In [ ]:

# Speed distribution of an ideal gas (molecular dynamics)
# With classical physics
# ---- gandreoliva.org ----

from vpython import *

# With plain classical mechanics

scene.center = vec(0.5,0.5,0.5)
scene.range = 1
L = 1
R = 0.03
T = 300 # <---- TEMPERATURE
dt = 2*1e-5
t = 0
kB = 1.4E-23
He_mass = 4E-3/6E23

# Average momentum of the particles (equipartition theorem)
pavg = sqrt(2*He_mass*1.5*kB*T) # p**2/(2m) = (3/2)*kB*T

outer_box = box(pos=vec(L/2.,L/2.,L/2.),size=vec(L,L,L),opacity=0.2)

# Detection and handling of collisions with the walls
def detect_handle_collision_wall(particle):
    # we check for velocity. Has the particle collided yet?
    if (particle.pos.x - particle.radius) < 0 and particle.vel.x < 0:
        particle.vel.x *=-1
    if (particle.pos.x + particle.radius) > L and particle.vel.x > 0:
        particle.vel.x *= -1
    if (particle.pos.y - particle.radius) < 0 and particle.vel.y < 0:
        particle.vel.y *=-1
    if (particle.pos.y + particle.radius) > L and particle.vel.y > 0:
        particle.vel.y *= -1
    if (particle.pos.z - particle.radius) < 0 and particle.vel.z < 0:
        particle.vel.z *= -1
    if (particle.pos.z + particle.radius) > L and particle.vel.z > 0:
        particle.vel.z *= -1


def handle_collision_pair(particle1,particle2):
    """
    Collision of hard spheres. Based on conservation of kinetic
    energy and momentum. Described in {W Krauth 2006 Statistical
    Mechanics: Algorithms and Computations, S 2.1.1}

    Warning: masses of particle1 and particle2 must be the same.

    The basic idea is: one may decompose the velocity vector in
    parallel and perpendicular components (with respect to the
    tangent plane that contains the contact point). Momentum is
    exchanged only in the perpendicular direction. In that
    direction, conservation of kinetic energy and momentum imply
    that the particles exchange their velocities
    (v1f = v2i; v2f = v2i), which can also be viewed as
    v1f = v1i + -(v1i-v2i)
    v2f = v2i + (v1i-v2i)
    """
    dx = particle1.pos - particle2.pos
    eperp = dx/mag(dx)
    dv = particle1.vel - particle2.vel

    particle1.vel += -eperp*(dv.dot(eperp))
    particle2.vel += eperp*(dv.dot(eperp))

def generate_random_p(pavg):
    # Actually, the initial state doesn't matter.
    # Equilibrium should be reached anyways.
    theta = pi*random()
    phi = 2*pi*random()
    px = pavg*sin(theta)*cos(phi)
    py = pavg*sin(theta)*sin(phi)
    pz = pavg*cos(theta)
    return vec(px,py,pz)

def random_uniform(a,b):
    return a+(b-a)*random()

# NUMBER OF PARTICLES
Num_particles = 100

particles = []
for i in range(Num_particles):
    particles.append(sphere(radius=R,
                            pos=vec(random_uniform(0.0+R,L/2-R),random_uniform(0.0+R,L-R),random_uniform(0.0+R,L-R)),\
                            # PRUEBE colocar una velocidad constante, ej: vel(200,0,0)
                            vel=generate_random_p(pavg)/He_mass,\
                            mass=He_mass\
                            ))

# Histogram
start = 0
end = 3000
bin_size = 100
nbins = (end-start)//100
gg = graph( width=500, height=0.4*500, xmax=3000, xtitle='speed, m/s', ytitle='Number of atoms', ymax=0.2*Num_particles)
histogram = gvbars(delta=bin_size)
data = []
ticker = 0
# Theoretical curve
dv = 10

# THEORETICAL PREDICTION: BOLTZMANN DISTRIBUTION
theory = gcurve( color=color.blue )
for v in range(0,3001+dv,dv):
    theory.plot( v, (bin_size/dv)*Num_particles*4*pi*((He_mass/(2*pi*kB*T))**1.5) *exp(-0.5*He_mass*(v**2)/(kB*T))*(v**2)*dv )


# MAIN SIMULATION
scene.waitfor("click")
while True:
    rate(25)
    for i,particle in enumerate(particles):
        # NEWTON'S LAWS
        particle.pos += particle.vel*dt

        # Collisions with the walls
        detect_handle_collision_wall(particle)

        # Collisions of this particle with others
        others = list(particles)
        for j in range(i): # This is to avoid double counting (i<j)
            other = others[j]
            dist = mag(particle.pos-other.pos)
            if dist < 2*R:
                handle_collision_pair(particle,other)

    # Histogram
    ticker += 1
    if ticker > 25: #Makes the refresh rate of the histogram slower
        ticker = 0
        histogram.delete()
        for k in range(nbins):
            bin_tot = 0
            for particle in particles:
                speed = mag(particle.vel)
                if speed > k*bin_size and speed < (k+1)*bin_size:
                    bin_tot += 1
            histogram.plot(k*bin_size,bin_tot)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>